<a href="https://colab.research.google.com/github/valiarw/recommender/blob/main/Latihan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages

In [87]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score

## Data Preprocessing

In [88]:
data = pd.read_csv('df.csv')
data.head()

,user_id,soal_id,hasil,materi_id,tingkat_kesulitan,nama_materi
0,1.0,101.0,benar,1.0,1.0,Pengantar Python
1,2.0,102.0,salah,1.0,1.0,Pengantar Python
2,3.0,103.0,salah,1.0,2.0,Pengantar Python
3,4.0,104.0,benar,1.0,2.0,Pengantar Python
4,5.0,105.0,salah,1.0,3.0,Pengantar Python


## Create Data

In [89]:
# Dataset Soal
data_soal = {
    'soal_id': [i for i in range(101,121)],
    "materi_id": ['1','1','1','1','1','1','2','2','2','2','2','2','2','3','3','3','3','3','3','3'],
    "tingkat_kesulitan": ['1','1','2','2','3','3','1','1','1','2','2','3','3','1','1','1','2','2','3','3']
    }
data_soal = pd.DataFrame(data_soal)
data_soal

,soal_id,materi_id,tingkat_kesulitan
0,101,1,1
1,102,1,1
2,103,1,2
3,104,1,2
4,105,1,3
5,106,1,3
6,107,2,1
7,108,2,1
8,109,2,1
9,110,2,2


In [90]:
# Dataset Jawaban User
data_jawaban = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 1, 2, 3, 2, 1, 2, 3, 1, 3, 2],
    'soal_id': [i for i in range(101, 121)], # Adjust the range to match the length of other lists
    'hasil': ['benar', 'salah', 'salah', 'benar', 'benar', 'salah', 'salah', 'benar', 'benar', 'salah',
              'salah', 'salah', 'benar', 'salah', 'benar', 'benar', 'benar', 'salah', 'salah', 'benar']
}

# Membuat DataFrame
data_jawaban = pd.DataFrame(data_jawaban)

# Menampilkan hasil
print(data_jawaban)

    user_id  soal_id  hasil
0         1      101  benar
1         1      102  salah
2         1      103  salah
3         2      104  benar
4         2      105  benar
5         2      106  salah
6         3      107  salah
7         3      108  benar
8         3      109  benar
9         3      110  salah
10        1      111  salah
11        2      112  salah
12        3      113  benar
13        2      114  salah
14        1      115  benar
15        2      116  benar
16        3      117  benar
17        1      118  salah
18        3      119  salah
19        2      120  benar


In [91]:
# Dataset Materi
data_materi = {
    'materi_id': ["1", "2", "3"],
    'nama_materi': ['Pengantar Python', 'Struktur Data', 'Fungsi dan Modul']
}
data_materi = pd.DataFrame(data_materi)
data_materi

,materi_id,nama_materi
0,1,Pengantar Python
1,2,Struktur Data
2,3,Fungsi dan Modul


In [92]:
# Gabungkan data soal dengan jawaban
df = pd.merge(data_jawaban, data_soal, on='soal_id')
df = pd.merge(df, data_materi, on='materi_id')
# Menyimpan DataFrame ke file CSV
df.to_csv('df.csv', index=False)

## Fitur dan Label
Fitur yang digunakan adalah rata-rata kesalahan per materi (kesalahan) dan tingkat kesulitan rata-rata soal di materi tersebut (tingkat_kesulitan).
Label adalah apakah user perlu mengulang materi atau tidak (1 jika perlu mengulang, 0 jika tidak).

In [93]:
# Convert 'tingkat_kesulitan' column to numeric
df['tingkat_kesulitan'] = pd.to_numeric(df['tingkat_kesulitan'])

# Buat kolom baru 'jumlah_salah' dengan nilai 1 jika hasil 'salah', 0 jika 'benar'
df['jumlah_salah'] = df['hasil'].apply(lambda x: 1 if x == 'salah' else 0)

# Hitung rata-rata kesalahan dan tingkat kesulitan untuk setiap user per materi
fitur_data = df.groupby(['user_id', 'materi_id']).agg({
    'jumlah_salah': 'mean',  # Rata-rata kesalahan per materi
    'tingkat_kesulitan': 'mean'  # Rata-rata tingkat kesulitan soal yang dikerjakan user
}).reset_index()

# Label: jika rata-rata kesalahan > 0.5, maka materi dianggap perlu diulang (1)
fitur_data['review_kembali'] = fitur_data['jumlah_salah'].apply(lambda x: 1 if x > 0.5 else 0)

# Tampilkan data yang sudah diolah
print(fitur_data)

   user_id materi_id  jumlah_salah  tingkat_kesulitan  review_kembali
0        1         1      0.666667           1.333333               1
1        1         2      1.000000           2.000000               1
2        1         3      0.500000           1.500000               0
3        2         1      0.333333           2.666667               0
4        2         2      1.000000           3.000000               1
5        2         3      0.333333           1.666667               0
6        3         2      0.400000           1.600000               0
7        3         3      0.500000           2.500000               0


## Melatih Model

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Fitur dan label
X = fitur_data[['jumlah_salah', 'tingkat_kesulitan']]  # Fitur yang digunakan
y = fitur_data['review_kembali']  # Label

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Buat dan latih model Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Prediksi di data uji
y_pred = model.predict(X_test)

# Evaluasi model
akurasi = accuracy_score(y_test, y_pred)
print(f'Akurasi model: {akurasi * 100:.2f}%')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Akurasi model: 33.33%
[[1 0]
 [2 0]]
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Evaluasi Model

In [95]:
# Evaluasi menggunakan metrik yang lebih mendalam
akurasi = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Tampilkan hasil evaluasi
print(f'Akurasi Model: {akurasi * 100:.2f}%\n')
print('Confusion Matrix:')
print(conf_matrix)
print('\nLaporan Klasifikasi:')
print(class_report)


Akurasi Model: 33.33%

Confusion Matrix:
[[1 0]
 [2 0]]

Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Rekomendasi berdasarkan model

In [96]:
# Fungsi untuk memberikan rekomendasi berdasarkan data user baru
def rekomendasi_materi(user_data, model):
    """
    Fungsi untuk memberikan rekomendasi apakah user perlu mengulang materi.
    user_data: Data performa user (DataFrame)
    model: Model machine learning yang sudah dilatih
    """
    # Ambil fitur yang diperlukan
    fitur_baru = user_data[['jumlah_salah', 'tingkat_kesulitan']]

    # Prediksi menggunakan model
    prediksi = model.predict(fitur_baru)

    # Tambahkan kolom prediksi ke dalam data
    user_data['review_kembali'] = prediksi

    # Tampilkan rekomendasi
    rekomendasi = user_data[user_data['review_kembali'] == 1]
    if not rekomendasi.empty:
        print("User perlu mengulang materi berikut:")
        for idx, row in rekomendasi.iterrows():
            print(f"- Materi ID: {row['materi_id']} (Rata-rata Kesalahan: {row['jumlah_salah']:.2f})")
    else:
        print("User tidak perlu mengulang materi apa pun.")

    return user_data


## Prediksi dengan data baru

In [97]:
# Simulasi data performa user baru
data_user_baru = {
    'user_id': [4, 4, 4],
    'materi_id': [101, 102, 103],
    'jumlah_salah': [0.6, 0.3, 0.7],  # Rata-rata kesalahan per materi
    'tingkat_kesulitan': [2, 1.5, 2.5]  # Rata-rata tingkat kesulitan soal yang dikerjakan
}

# Buat DataFrame dari data user baru
df_user_baru = pd.DataFrame(data_user_baru)

# Gunakan model untuk memberikan rekomendasi pada user baru
hasil_rekomendasi = rekomendasi_materi(df_user_baru, model)
print("\nHasil prediksi dan rekomendasi untuk user baru:")
print(hasil_rekomendasi)


User tidak perlu mengulang materi apa pun.

Hasil prediksi dan rekomendasi untuk user baru:
   user_id  materi_id  jumlah_salah  tingkat_kesulitan  review_kembali
0        4        101           0.6                2.0               0
1        4        102           0.3                1.5               0
2        4        103           0.7                2.5               0
